# 6장 데이터 로딩과 저장, 파일 형식

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [ ]:
!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv")
df

    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [ ]:
pd.read_csv("examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names)

    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [ ]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [ ]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [ ]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [ ]:
result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [ ]:
#pd.options.display.max_rows = 10
result = pd.read_csv("examples/ex6.csv")

result = pd.read_csv("examples/ex6.csv", nrows=5)
result

### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [ ]:
# !cat examples/ex6.csv
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64') #Series 객체 생성

for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0) 

tot = tot.sort_values(ascending=False)
tot
# tot[:10]

### 1.2. csv 텍스트 파일 저장하기
 판다스에서 csv파일을 저장하기 : df.to_csv(path, encoding = 'utf-8')함수 사용
path : 파일 시스템에서의 위치,url,파일객체를 나타내는 문자열
encoding : 유니코드 인코딩 종류를 지정

In [ ]:
data = pd.read_csv("examples/ex5.csv")
data # dataframe에 데이터 저장
data1 = data[['a','b','d']] # 인덱싱 : 데이터 프레임의 일부 데이터만 추출해서 데이터프레임 생성 
data1
data1.to_csv("examples/out.csv") # 데이터프레임.to_csv(저장할 파일명)

# 2. HTML 데이터
## 2.1 웹 스크래핑 - read_html()

* pd.read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [ ]:
import pandas as pd

tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
print(type(tables))
# len(tables)

failures = tables[0] # 인덱싱 
# print(type(failures))
failures.head()

In [ ]:
# 1) 컬럼명 수정하기
# failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

In [ ]:
# 2) 데이터 변환 : 시간/날짜 문자열 -> datetime 객체 변환
close_timestamps = pd.to_datetime(failures["ClosingDate"])
close_timestamps.head()
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [ ]:
!pip install html5lib  lxml

In [ ]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
type(df_list)
len(df_list)
# 데이터프레임 확인
# for df in df_list:
#     print(df)
df_list[1]

In [ ]:
# 데이터프레임 탐색
df = df_list[1]
# 결측치 처리 - 종목명
# 조건색인을 이용해서 불리언 인덱싱으로 새로운 데이터프레임 생성
#1. 조건 색인
df['종목명'].notnull() # notna()
#2. 조건색인을 데이터프레임에 넣어주기 -true 값만 추출 
cond = df['종목명'].notnull()
data = df[cond]
data
data.head()
# 최종 결과를 데이터프레임 형태로 저장 - df.to_csv(파일명)
data.to_csv("Naver_kospi.csv")

## 2.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 

In [ ]:
# 1. requests 모듈 설치하기
# !pip install requests
# 2. requests 모듈 임포트하기
import requests
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
print(resp.status_code)
#4. 웹페이지 내용 텍스트 확인
resp.text
#원하는 데이터 처리 방법 
# 5. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
type(data)
len(data)
#6. json 내용 확인 
data[0]
#7. json의 딕셔너리
data[0].keys() 
# # 8. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()
# 데이터프레임을 저장
issues.to_csv("pandas_issues.csv")

### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [ ]:
import requests
# url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}

# response = requests.get(url)

response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

## 2.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    # 웹 페이지 정보를 원하는 데이터 정보 추출
    #1. html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')

    #2. 첫 번째 제목 한개 선택하여 텍스트 가져오기
    # title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # print(title.get_text())

    # 첫 번째 웹페이지의 모든 제목들을 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    print(type(titles))
    for title in titles:
        print(title.get_text()) # 제목 텍스트 가져오기
else : 
    print(response.status_code)



## 2.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제] 크롬 드라이버를 이용하여 타겟 웹사이트 실행하기 

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome()
# 타켓 웹페이지 열기
url = "https://www.google.com"
driver.get(url)

NoSuchDriverException: Message: Unable to obtain driver for chrome using Selenium Manager.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


### [문제] Selenium을 이용하여 타겟 웹사이트 실행하기 /검색어

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1. Chrome 브라우저 실행
driver = webdriver.Chrome()

#2. 타켓 웹페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(2)

try:
    # 1. 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 2. 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    # 3. Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 4. 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")

    # 검색 결과의 제목(<h3></h3>)과 URL(<a href="..."></a>) 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

# finally:
#     # 브라우저 창 닫기
#     driver.quit()


### [응용문제] : 시가총액이 높은 상위 500개 기업의 데이터를 가져오기
* 1~50 페이지 중 10개 페이지만 데이터 읽어오기 
* 10개 페이지를 하나의 데이터로 합치기
* 위 결과를 csv 파일로 저장

In [ ]:
from selenium import webdriver
import pandas as pd
import time

In [ ]:
# Chrome 브라우저를 열고 웹 페이지로 이동
driver = webdriver.Chrome()

# 시가 총액 상위 500개 기업 데이터를 가져올 url
url = "https://finance.naver.com/sise/sise_market_sum.naver?&page="

#10개 페이지의 데이터를 저장할 리스트
kospi500 = []

#10개 페이지에 대해 반복하여 데이터 수집
for num in range(1, 11): # 페이지 번호 제어
    full_url = url +str(num)
    driver.get(full_url) # 페이지로 이동
    time.sleep(2) #페이지가 로드될 때까지 기다림
    html = driver.page_source #페이지의 html가져오기
    data = pd.read_html(html) #html을 데이터프레임으로 읽기 시도
    df = data[1] # 두 번째 테이블에 종목 데이터가 있음
    df = df[df['종목명'].notnull()] #조건 색인으로 결측치 처리
    kospi500.append(df) #선택된 데이터를 리스트에 추가

#10개 패이지의 데이터를 하나의 데이터프레임으로 합치기
combined_df = pd.concat(kospi500)

# 결과 출력
combined_df .to_csv("combined_kospi.csv")

# #드라이버 닫기 
# driver.quit()


In [ ]:
from selenium import webdriver
from bs4 import BeutifulSoup
import pandas as ppd
import time

In [ ]:
# chrome 웹 드라이버 실행
driver = webdriver.Chrome()

In [ ]:
#페이지 로딩 시간
time.sleep(2)

In [ ]:
#targer 웹페이지 url - 사전 작업 
# 삼성전자 일별 시세 정보 페이지
samsung_url = "https://finance.naver.com/item/side_day.naver?code=005930"

#페이지 요청
driver.get(samsung_url)

In [ ]:
try:
    # 웹 페이지의 html 가져옹기
    html = driver.page_source
    df = pd.read_html
    print(df)
except Exception as e:
    print("Network Error")
finally:
    driver.quit(()